In [118]:
import pandas as pd
import numpy as np


drinks = [
    "Bavaria S", "Bavaria L", "Witte S", "Witte L", "Ultje S", "Ultje L",
    "Bavaria 0.0 S", "Bavaria 0.0 L", "Pepsi", "Pepsi Max", "Sisi", "7 Up",
    "Ice Tea", "Trappe Blond", "Trappe Dubbel", "Trappe Isidor", "Tonic",
    "Apple Juice", "Green Ice Tea", "Ginger Beer", "Energy", "Gin Tonic",
    "Rum Cola", "Hennesey Apple", "Vodka Energy", "Vodka", "Rum", "Gin", "Hennesey"
]


np.random.seed(42)  
quantities = np.random.randint(50, 600, size=len(drinks))  


df_new = pd.DataFrame({"Drink": drinks, "Quantity Sold": quantities})


df_new.to_excel("generated_drink_sales.xlsx", index=False)

print("Excel file 'generated_drink_sales.xlsx' created successfully.")


Excel file 'generated_drink_sales.xlsx' created successfully.


In [1]:
import pandas as pd
import math

# Drink-to-keg conversion dictionary

conversion_rates = {
    "Bavaria S": 66.6, #small beers per keg (small beer=0.3 L, keg=20 L)
    "Bavaria L": 40, #large beers per keg (large beer 0.5 L)
    "Witte S": 66.6,
    "Witte L": 40,
    "Ultje S": 66.6, #could consider in the future that Ultje & Witte make more foam, so less cups per keg
    "Ultje L": 40,
    "Bavaria 0.0 S": 66.6,
    "Bavaria 0.0 L": 40,
    "Pepsi": 100, #soft drink=0.3L, soft drink keg=30L
    "Pepsi Max": 100,
    "sisi": 100,
    "7 Up": 100,
    "Ice Tea": 100, #same consideration as Ultje
    "Trappe Blond": 1, #bottles are counted with a 1:1 ratio, for future I could work with crates
    "Trappe Dubbel": 1,
    "Trappe Isidor": 1,
    "Tonic": 1,
    "Apple Juice": 1,
    "Green Ice Tea": 1,
    "Ginger Beer": 1,
    "Energy": 1,
    "Gin Tonic": 1, #for mixed drinks I don't care about rates because they work differently
    "Rum Cola": 1,
    "Hennesey Apple": 1,
    "Vodka Energy": 1,
    "Vodka": 33.3, #any shot=0.03 L, bottle=1 L
    "Rum": 33.3,
    "Gin": 33.3,
    "Hennesey": 33.3
}

In [ ]:
#Load excel file

def calculate_bottles_needed(file_path):
    df = pd.read_excel(file_path)
    quantity_sold = {}  # Dictionary for drink sold
   
    # all drinks sold are in a dictionary
    
    for _, row in df.iterrows():
        drink = row["Drink"]
        quantity = row["Quantity Sold"]
        quantity_sold[drink] = quantity_sold.get(drink, 0) + quantity

    # Update quantities for mixed drinks
    
    if "Gin Tonic" in quantity_sold:
        quantity_sold["Gin"] = quantity_sold.get("Gin", 0) + quantity_sold["Gin Tonic"]
        quantity_sold["Tonic"] = quantity_sold.get("Tonic", 0) + quantity_sold["Gin Tonic"]
        
    if "Rum Cola" in quantity_sold:
        quantity_sold["Rum"] = quantity_sold.get("Rum", 0) + quantity_sold["Rum Cola"]
        quantity_sold["Pepsi"] = quantity_sold.get("Pepsi", 0) + quantity_sold["Rum Cola"]

    if "Hennesey Apple" in quantity_sold:
        quantity_sold["Hennesey"] = quantity_sold.get("Hennesey", 0) + quantity_sold["Hennesey Apple"]
        quantity_sold["Apple Juice"] = quantity_sold.get("Apple Juice", 0) + quantity_sold["Hennesey Apple"]

    if "Vodka Energy" in quantity_sold:
        quantity_sold["Vodka"] = quantity_sold.get("Vodka", 0) + quantity_sold["Vodka Energy"]
        quantity_sold["Energy"] = quantity_sold.get("Energy", 0) + quantity_sold["Vodka Energy"]

        bottle_orders = {}  # Dictionary for bottle/keg to order

            
    #actual stuff    
            
            
            
    # Calculate bottles/keg needed based on updated quantities sold
    
    for drink, quantity in quantity_sold.items():
        if drink in conversion_rates:
            rate = conversion_rates[drink]
            
        if drink.endswith(" S") or drink.endswith(" L"): # Beer S/L case
                base_drink = drink[:-2]  # Remove "S" or "l"
                liters_consumed = quantity * (20/rate) #20 assuming all beer keg are 20 L
                bottle_orders[base_drink] = bottle_orders.get(base_drink, 0) + (liters_consumed / 20.0)
        
        else:  # Normal drinks
            bottle_orders[drink] = bottle_orders.get(drink, 0) + (quantity / rate)            
            
    return {drink: int(math.ceil(bottles)) for drink, bottles in bottle_orders.items() if drink not in ["Gin Tonic", "Rum Cola", "Hennesey Apple", "Vodka Energy"]}
#rounded up just to be sure





#final step
file_path = r"C:\Users\111946\Desktop\Poppodium Project\generated_drink_sales.xlsx"
print("Number of kegs or bottles to be bought: \n \n", calculate_bottles_needed(file_path))


#possible update:

# - use actual real data, both # of drinks and quantities per keg/bottle
# - single drinks (ginger beer/tonic/la trappe bottles) should be
#   counted in crates, not in single bottles
# - consider waste/foam/errors
# - take into account music genres and predict consumes
# - make the output more beautiful

Number of kegs or bottles to be bought: 
 
 {'Bavaria': 13, 'Witte': 9, 'Ultje': 4, 'Bavaria 0.0': 16, 'Pepsi': 5, 'Pepsi Max': 4, 'Sisi': 6, '7 Up': 2, 'Ice Tea': 5, 'Trappe Blond': 149, 'Trappe Dubbel': 180, 'Trappe Isidor': 358, 'Tonic': 719, 'Apple Juice': 1050, 'Green Ice Tea': 463, 'Ginger Beer': 435, 'Energy': 604, 'Vodka': 14, 'Rum': 16, 'Gin': 26, 'Hennesey': 19}
